# Environment Check

Click the button below to print Python, PyTorch, CUDA, `nvidia-smi`, and run a tiny GPU test.

In [ ]:
import os, shutil, subprocess, sys
import torch
from ipywidgets import Button, Output, VBox
from IPython.display import display

out = Output()

def _run(cmd, timeout=20):
    try:
        res = subprocess.run(
            cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
            text=True, timeout=timeout, check=False
        )
        return res.stdout.strip()
    except Exception as e:
        return f"<error: {e}>"

def _fmt_gb(x):
    try:
        return f"{float(x):.2f}"
    except Exception:
        return str(x)

def check_env(_=None):
    out.clear_output()
    with out:
        print("=== Environment Check ===")
        print(f"Python       : {sys.version.split()[0]}")
        print(f"PyTorch      : {torch.__version__}")
        print(f"CUDA (Torch) : {getattr(torch.version, 'cuda', None)}")
        print(f"CUDA available via Torch: {torch.cuda.is_available()}")

        if torch.cuda.is_available():
            try:
                name = torch.cuda.get_device_name(0)
                total_gb = torch.cuda.get_device_properties(0).total_memory / (1024**3)
                print(f"GPU name     : {name}")
                print(f"GPU memory   : {_fmt_gb(total_gb)} GB")
            except Exception as e:
                print(f"GPU props    : <error: {e}>")

        print("\n--- nvidia-smi ---")
        if shutil.which("nvidia-smi"):
            print(_run(["nvidia-smi"]))
        else:
            print("nvidia-smi not found in PATH")

        print("\n--- GPU matmul test ---")
        if torch.cuda.is_available():
            try:
                x = torch.randn(1000, 1000, device="cuda")
                y = x @ x
                print("GPU matmul OK; result mean:", y.mean().item())
            except Exception as e:
                print("GPU matmul failed:", e)
        else:
            print("CUDA not available; skipped matmul")

btn = Button(description="Run Environment Check", button_style="primary")
btn.on_click(check_env)

display(VBox([btn, out]))